Reads the training configuration yaml

Generate a PAT embedding model in `assets/models/PAT/`, with `.pth` checkpoints every 5 epochs

In [2]:
import os
import zipfile
import dotenv
import subprocess
import pandas as pd
from typing import Dict, Any
import json
import yaml
import time
import glob
import hashlib
import shutil
import logging
import pandas as pd


from PIL import Image
from transformers import pipeline
import torch

config_train = "config/UrbanElementsReID_train.yml"  # "config/UAM_containers.yml"

################ Probably nothing has to be modified from now on ################
with open(config_train, 'r') as f:
    hyperparams_train = yaml.load(f, Loader=yaml.BaseLoader)
model_path = os.path.join(hyperparams_train['LOG_ROOT'], hyperparams_train['LOG_NAME'])

logging.root.setLevel(logging.INFO)

if not torch.cuda.is_available():
    logging.warning("Where is your GPU dude?")

In [3]:
!nvidia-smi

Wed Apr 30 12:09:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  | 00000000:01:00.0 Off |                  Off |
| N/A   27C    P0              33W / 250W |      7MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
assert os.path.exists('assets'), 'are you are in the right folder?'
assert os.getcwd().endswith('PAT'), 'are you are in the right folder?'
if os.path.exists(model_path):
    shutil.rmtree(model_path)

!python train.py --config_file {config_train}

2025-04-30 12:09:57,454 PAT INFO: Saving model in the path :assets/models/PAT
2025-04-30 12:09:57,455 PAT INFO: Namespace(config_file='config/UrbanElementsReID_train.yml', opts=[], local_rank=0)
2025-04-30 12:09:57,455 PAT INFO: Loaded configuration file config/UrbanElementsReID_train.yml
2025-04-30 12:09:57,455 PAT INFO: 
MODEL:
  PRETRAIN_CHOICE: 'imagenet'
  #PRETRAIN_PATH: "../../.cache/torch/hub/checkpoints" # root of pretrain path
  PRETRAIN_PATH: "assets/models" 
  IF_LABELSMOOTH: 'on'
  IF_WITH_CENTER: 'no'
  NAME: 'part_attention_vit'
  NO_MARGIN: True
  DEVICE_ID: ('0')
  TRANSFORMER_TYPE: 'vit_base_patch16_224_TransReID'
  STRIDE_SIZE: [16, 16]

INPUT:
  SIZE_TRAIN: [256,128]
  SIZE_TEST: [256,128]
  REA:
    ENABLED: False
  PIXEL_MEAN: [0.5, 0.5, 0.5]
  PIXEL_STD: [0.5, 0.5, 0.5]
  LGT: # Local Grayscale Transfomation
    DO_LGT: True
    PROB: 0.5

DATASETS:
  TRAIN: ('UrbanElementsReID',)
  TEST: ('UrbanElementsReID',)
  #ROOT_DIR: ('../../data') # root of datasets
  #RO

In [14]:
files = glob.glob(os.path.join(model_path, "part_attention_vit_*.pth"))
max_epoch = max([int(f.split('_')[-1].split('.')[0]) for f in files])
assert max_epoch > 0
assert max_epoch == int(hyperparams_train['SOLVER']['MAX_EPOCHS'])
assert os.path.exists(os.path.join(model_path, f'part_attention_vit_{max_epoch}.pth'))
logging.info(f'max_epoch={max_epoch}')

INFO:root:max_epoch=60
